In [1]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.device_count(), device

(0, device(type='cpu'))

In [2]:
import os
import sys
IS_GCOLAB = 'google.colab' in sys.modules

In [3]:
from pathlib import Path

_base_libpath = Path(
    "/content/drive/MyDrive" if IS_GCOLAB else f"{os.environ['HOME']}"
) / "_self_research/24.09.world_cuisine"

print(_base_libpath)

assert _base_libpath.exists()

os.chdir(_base_libpath)

/Users/afaji/_self_research/24.09.world_cuisine


In [4]:
if IS_GCOLAB:
    raise NotImplementedError()
    # from google.colab import drive
    # drive.mount('/content/drive')
    # # os.environ.setdefault("", "1")
    # # os.system("pip install transformers==4.33.1")
    # os.system("pip install transformers sudachitra accelerate")
    # os.system("pip install langchain tiktoken openai")
    # os.system("pip install -U langchain-community")
    # os.system("pip install -U datasets pyarrow")

else:
    # raise NotImplementedError()
    print(">>> Not on Google Colab <<<")

>>> Not on Google Colab <<<


---

In [5]:
# import datasets
# from datasets import load_dataset
# from torch.utils.data import DataLoader
# from sentence_transformers import SentenceTransformer

import os
import requests
import pickle
from PIL import Image
from glob import glob
from pathlib import Path
import numpy as np
import pandas as pd

import time
from tqdm import tqdm

import re
import json
from collections import Counter

In [6]:
df = pd.read_csv("WorldCuisines - Food.csv")
print(f"Cuisine counts: {len(df):6,}")

df = df.loc[df.loc[:,'Image1 name / URL'].notna()].reset_index(drop=True)
print(f"  w/ imgs only: {len(df):6,}")

Cuisine counts:  3,733
  w/ imgs only:  1,919


In [7]:
# df = pd.read_csv("WorldCuisines - Food.240919.csv")

In [8]:
fps = []
for _ in range(1,9):
    wew = df.loc[:,f"Image{_} name / URL"].dropna()
    fps.extend(wew.tolist())
fps = pd.Series(fps)

In [9]:
wew = fps.map(lambda t: t[:-len("?download")].lower() if t.endswith("?download") else t.lower())

for x in wew:
    if not (x.endswith(".jpg") | x.endswith(".jpeg")
    | x.endswith(".png") | x.endswith(".gif") | x.endswith(".tif")):
        print(x)

assert not wew[~(
    wew.str.endswith(".jpg") | wew.str.endswith(".jpeg")
    | wew.str.endswith(".png") | wew.str.endswith(".gif") | wew.str.endswith(".tif")
)].tolist()
# print(len(wew), len(wew.unique()))

https://upload.wikimedia.org/wikipedia/commons/thumb/0/08/scones_cream_jam.jpg/1440px-scones_cream_jam.jpg?downloadd
https://upload.wikimedia.org/wikipedia/commons/thumb/c/cc/%d0%9f%d1%8b%d1%88%d0%ba%d0%b8.jpg/1600px-%d0%9f%d1%8b%d1%88%d0%ba%d0%b8.jpg?20170727221718
https://upload.wikimedia.org/wikipedia/commons/7/79/faworki_%28close-up%29.jpg?20070219221018
https://upload.wikimedia.org/wikipedia/commons/thumb/c/ca/food_013_white.jpg/1600px-food_013_white.jpg?20160305082454
https://upload.wikimedia.org/wikipedia/commons/thumb/f/f1/khrenovina-sauce.jpg/1600px-khrenovina-sauce.jpg?20170807205645
https://upload.wikimedia.org/wikipedia/commons/thumb/8/8d/bejgli2.jpg/1600px-bejgli2.jpg?20071231132011
https://upload.wikimedia.org/wikipedia/commons/thumb/c/cc/trichiurus_lepturus_sushi.jpg/1600px-trichiurus_lepturus_sushi.jpg?20120713234225
https://upload.wikimedia.org/wikipedia/commons/a/a3/katsusando6515.jpg?20101025191551
https://upload.wikimedia.org/wikipedia/commons/thumb/7/71/yakisobapan

AssertionError: 

In [10]:
print(f"Image Count: {len(fps):,}")
fps = fps[~wew.duplicated()].reset_index(drop=True)
print(f"  Unique # : {len(fps):,}")

Image Count: 5,365
  Unique # : 5,308


In [38]:
# as of 240919

In [11]:
from bs4 import BeautifulSoup
import csv
import os
from tqdm import tqdm

#di-cache biar gak extract web yg udah pernah ke-cache sebelomnya
cache_data_dict = {}

if os.path.exists('license_cache_data.csv'):
    with open('license_cache_data.csv', 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            cache_data_dict[row[0]] = row[1]


def extract_website(url):
    if pd.isna(url):
        return None
    else:
        x = re.search("https?://(.+?)/", url)
        if not x:
            return None
        else:
            return x.group(1)

# convert to common wikimedia url to extract url
def convert_media_url_to_commons_page(media_url):
    # Extract the file name
    parts = media_url.split("/")
    file_name_1 = parts[-1]
    file_name_2 = parts[-2]  # The last part of the URL is the file name
    # check if [number]px in the file nbame then extract file_name_2
    if re.match(r"\d+px", file_name_1):
        file_name = file_name_2
    else:
        file_name = file_name_1

    # Construct the Wikimedia Commons page URL
    commons_page_url = f"https://commons.wikimedia.org/wiki/File:{file_name}"
    return commons_page_url

# license checker:
def get_wikimedia_image_license(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    license_short = soup.find("span", {"class": "licensetpl_short"})
    if license_short:
        return license_short.text.strip()
    else:
        return None

invalid_url = 0
invalid_license = 0

extracted_license = []

for url in tqdm(fps):
    base_url = extract_website(url)
    if not base_url or base_url != "upload.wikimedia.org":
        invalid_url += 1
        extracted_license.append((url, None))
        continue
    
    convert_url = convert_media_url_to_commons_page(url)
    
    if url in cache_data_dict:
        license = cache_data_dict[url]
    else:
        license = get_wikimedia_image_license(convert_url)
        if license != None:
            cache_data_dict[url] = license
        else:
            print("Found invalid license: ", url)

    with open('license_cache_data.csv', 'w') as f:
        writer = csv.writer(f)
        for key, value in cache_data_dict.items():
            writer.writerow([key, value])

    extracted_license.append((url, license))

for e in extracted_license:
    print(e)


 19%|██████████████████▋                                                                                 | 992/5308 [07:09<45:37,  1.58it/s]

Found invalid license:  https://upload.wikimedia.org/wikipedia/commons/thumb/0/01/Amish_shoo-fly_pies%2C_made_from_a_gooey_molasses-based_filling_and_sometimes_topped_with_chocolate_cream._Lancaster%2C_Pennsylvania_LCCN2011634806.tif/lossy-page1-1348px-Amish_shoo-fly_pies%2C_made_from_a_gooey_molasses-based_filling_and_sometimes_topped_with_chocolate_cream._Lancaster%2C_Pennsylvania_LCCN2011634806.tif.jpg?download


 25%|████████████████████████▉                                                                          | 1336/5308 [10:56<52:55,  1.25it/s]

Found invalid license:  https://upload.wikimedia.org/wikipedia/en/thumb/4/4b/IRU.JPG/1440px-IRU.JPG?download


 25%|█████████████████████████                                                                          | 1346/5308 [11:05<53:06,  1.24it/s]

...

Found invalid license:  https://upload.wikimedia.org/wikipedia/commons/thumb/b/bd/Pot-au-feu_Vegetable_ready_for_cooking.png/1440px-Pot-au-feu_Vegetable_ready_for_cooking.png?download


 90%|████████████████████████████████████████████████████████████████████████████████████████▊          | 4764/5308 [52:52<08:22,  1.08it/s]

Found invalid license:  https://upload.wikimedia.org/wikipedia/en/thumb/b/b3/AlfajorBaguioKikoBautista.jpg/1440px-AlfajorBaguioKikoBautista.jpg?download


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 5308/5308 [1:01:52<00:00,  1.43it/s]

('rawon.jpg', None)
('soto_betawi.jpg', None)
('soto_bandung.jpg', None)
('soto_banjar.jpg', None)
('gulai.jpg', None)
('kare-kare.jpg', None)
('pindang_patin.jpg', None)
('chicken_tikka_masala.jpg', None)
('chicken_tikka.jpg', None)
('saag.jpg', None)
('dhansak.jpg', None)
('palak_paneer.jpg', None)
('dal_makhani.jpg', None)
('nasi_uduk.jpg', None)
('nasi_lemak.jpg', None)
('nasi_campur.png', None)
('nasi_goreng.jpg', None)
('paella.jpg', None)
('arros_negre.jpg', None)
('jollof_rice.jpg', None)
('biryani.jpg', None)
('khao_phat.jpg', None)
('hokkien_fried_rice.jpg', None)
('yin_yang_fried_rice.jpg', None)
('chahan.jpg', None)
('omurice.jpg', None)
('bokkeum_bap.jpg', None)
('aligue_fried_rice.jpg', None)
('java_rice.jpg', None)
('sinangag.jpg', None)
('yangzhou_fried_rice.jpg', None)
('donburi.jpg', None)
('akhni.jpg', None)
('akki_rotti.jpg', None)
('ampaw.jpg', None)
('appam.jpg', None)
('arancini.jpg', None)
('arem-arem.jpg', None)
('arroz_caldoso.jpg', None)
('arroz_chaufa.jpg', 

---

In [ ]:
# assert all(fps[fps.str.startswith("http")].str.startswith("https://"))

In [ ]:
def get_img_size(fp):
    headers = {'User-Agent': 'CoolBot/0.0 (https://example.org/coolbot/; coolbot@example.org)'}
    try:
        if fp.startswith("https://"):
            response = requests.get(fp, headers=headers, stream=True, timeout=8)
            response.raise_for_status()
            _img = response.raw
        else:
            _img = _base_libpath / "images" / fp
        image = Image.open(_img).convert('RGB')
        return image.size
    except Exception as e:
        # print(fp, e)
        return None

In [ ]:
from multiprocessing import Pool

In [ ]:
%%time
with Pool(processes=32) as pool:
    async_result = pool.map_async(get_img_size, fps.tolist(), 16)
    async_result.wait()

In [ ]:
sizes = async_result.get()

In [ ]:
len(sizes), len(list(filter(None, sizes))), f"{len(list(filter(None, sizes)))/len(sizes):.1%}"

In [ ]:
# Previous count when total images was 1871

---

In [ ]:
import seaborn as sns

In [ ]:
df = pd.DataFrame(list(filter(None, sizes)), columns=["width", "height"])

In [ ]:
df

In [ ]:
df["hue"] = ""

In [ ]:
g = sns.JointGrid(df, x="width", y="height", hue="hue")
g.plot_joint(sns.scatterplot, s=20, alpha=.1)
g.plot_marginals(sns.histplot, kde=True, linewidth=0, palette=["g"])

as of 240919

In [ ]:
g = sns.JointGrid(df.loc[df.width.le(3072)&df.height.le(2048)], x="width", y="height", hue="hue")
g.plot_joint(sns.scatterplot, s=30, alpha=.2)
g.plot_marginals(sns.histplot, kde=True, linewidth=.2, palette=["g"])

---
---
---

In [ ]:
df_all = pd.read_csv("WorldCuisines - Food.csv")
print(f"Cuisine counts: {len(df_all):6,}")
df_w_img = df_all.loc[df_all.loc[:,'Image1 name / URL'].notna()].reset_index(drop=True)
print(f"   has images : {len(df_w_img):6,}")
df_w_reg = df_all[df_all.loc[:,'Region1'].notna()].reset_index(drop=True)
print(f"   has Region1: {len(df_w_reg):6,}")

In [ ]:
print("--- No Images stats ---")
# print(f"Has Region1, w/o imgs. #: {sum(df_all.loc[:,'Region1'].notna())-len(df_w_img):6,}")
print(f"w/ Region1, w/o imgs.: {len(df_w_reg[df_w_reg.loc[:,'Image1 name / URL'].isna()]):6,}")
print(f"    =>  w/  Text Desc. # : {len(df_w_reg[df_w_reg.loc[:,'Text Description'].notna() & df_w_reg.loc[:,'Image1 name / URL'].isna()]):6,}")
print(f"    =>  w/o Text Desc. # : {len(df_w_reg[df_w_reg.loc[:,'Text Description'].isna()]):6,}")

In [ ]:
df_w_reg.iloc[:,16]

In [ ]:
print("NaN categories also got.\nNo Cuisine also got..\n:\\")
df_w_reg[df_w_reg.loc[:,"Coarse-grained categories"].isna()|df_w_reg.loc[:,"Cuisines"].isna()].iloc[:,:-17]

In [ ]:
print("Got categories and cuisines, but no description..")
df_w_reg[df_w_reg.loc[:,"Coarse-grained categories"].notna()&df_w_reg.loc[:,"Cuisines"].notna()&df_w_reg.loc[:,"Text Description"].isna()].iloc[:,:-17]

Let's check the stats

In [ ]:
df = df_w_reg[df_w_reg.loc[:,"Coarse-grained categories"].notna()&df_w_reg.loc[:,"Cuisines"].notna()&df_w_reg.loc[:,"Text Description"].notna()]
print(f"Somewhat OK count: {len(df)}")

In [ ]:
# @title
def parse_txt(t):
    if t is np.nan:
        return []
    t = t.replace('"Liberia"m"Cote D\'Ivoire"', '"Liberia", "Cote D\'Ivoire"')
    t = t.replace("''", '"')
    t = re.sub(r"[‘'’“]", '"', t)
    t = re.sub(r'", ([A-Z])', r'", "\1', t)
    t = re.sub(r'([a-z])\]', r'\1"]', t)
    t = re.sub(r'([a-z]), "', r'\1", "', t)
    t = (t
            .replace('" "', '", "')
            .replace('"."', '", "')
            .replace(", and ", ", ")
            .replace('d"Ivoire', "d'Ivoire")
            .replace('D"Ivoire', "D'Ivoire")
        )
    try:
        t = json.loads(t)
    except Exception as e:
        print(" >>> lho:", t)
        raise e
    return t


In [ ]:
entries = df.loc[:,[
    'Coarse-grained categories', 'Fine-grained categories', 'Cuisines',
    'Associated Cuisines', 'Area', 'Countries'
    ]]

In [ ]:
counters = {}
for c in range(6):
    counter = Counter([e.lower() for _ in entries.iloc[:,c].apply(parse_txt) for e in _])
    counters[entries.columns[c]] = counter
    print(entries.columns[c], f"  (total:  {sum(counter.values()):,})")
    print(counter, end="\n\n")

In [ ]:
print("List of countries and cuisines so far:")
print(*sorted(counters["Cuisines"].keys() | counters["Associated Cuisines"].keys() | counters["Countries"].keys()), sep="\n")

In [ ]:
for c, cnt in sorted((counters["Cuisines"] + counters["Countries"] + counters["Associated Cuisines"]).items()):
    print(c, cnt, sep='\t')